In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import json
import requests

import tweepy
from tweepy import Client

key = ""
secret = ""
token = ""

api = tweepy.Client(bearer_token=token, consumer_key=key, consumer_secret=secret, 
              return_type=requests.Response,
              wait_on_rate_limit=True)

In [35]:
already_processed = [1016773517587537920,
 837316817324556289,
 4307623643,
 844779426260791296,
 790017240733278208,
 913028718377005056,
 2855732920,
 1416121214,
 821215184660393986,
 1004764640486854656,
 352132252,
 1661404440,
 2928556470,
 2901953015,
1058130850578137089]

In [18]:
import os

already_processed = []
 
for name in os.listdir('bot'):
    if os.path.isdir(name):
        continue
    list_name = name.replace('.pkl','').replace('edgelist_2_edges_from_start_','')
    already_processed.append(int(list_name))

In [4]:
combined = pd.read_pickle('node_and_degree_list.pkl')
combined.head()

,twitter_id,followers_count,following_count,label
0,461277906,380850,2913,bot
1,808280162,23363,340,bot
2,1575510320,120435,108954,bot
3,2449235599,25581,16,bot
4,1113769100,73720,77585,bot


In [5]:
# Set follower/following threshold
thresh = 1000

print('BEFORE FILTERING')
print(f'Total accounts: {combined.shape[0]}')
nb = combined.loc[combined.label == 'bot'].shape[0]
nh = combined.loc[combined.label == 'human'].shape[0]
print(f'Bot accounts: {nb}')
print(f'Human accounts: {nh}')
print('____________________')


# Drop accounts that exceed threshold
small = combined.loc[combined.followers_count < thresh]
small = small.loc[small.followers_count < thresh]

print('AFTER FILTERING')
print(f'Total accounts: {small.shape[0]}')
nb = small.loc[small.label == 'bot'].shape[0]
nh = small.loc[small.label == 'human'].shape[0]
print(f'Bot accounts: {nb}')
print(f'Human accounts: {nh}')

BEFORE FILTERING
Total accounts: 2263
Bot accounts: 959
Human accounts: 1304
____________________
AFTER FILTERING
Total accounts: 458
Bot accounts: 193
Human accounts: 265


In [6]:
# Small df contains accounts meeting threshold criteria
small.head()

,twitter_id,followers_count,following_count,label
24,4307623643,59,4,bot
36,844779426260791296,768,1,bot
83,837316817324556289,182,6,bot
137,913028718377005056,87,147,bot
175,790017240733278208,19,47,bot


In [20]:
# Create curr_edgelist for a network including nodes up to 3 degrees of separation (3 links away)
def add_this_nodes_edgelist(this_acc, curr_edgelist):
    next_page_token = None
    visited.add(this_acc)

    action = 'followers'
    follow_list = []
    this_acc_num_following = []    
    this_acc_num_followers = []
    while True:
        # Get immediate followers/following of node and add these to curr_edgelist
        if action == 'followers':
            request = api.get_users_followers(id=this_acc, user_auth=False, 
                max_results=1000, pagination_token=next_page_token, user_fields=['public_metrics'])
        elif action == 'following':
            request = api.get_users_following(id=this_acc, user_auth=False, 
                max_results=1000, pagination_token=next_page_token, user_fields=['public_metrics'])

        data = request.json()

        # for each account following/followed by the originating account, record account id, 
        # num following, and num followed by
        try:
            for acc in data['data']:
                follow_list.append(acc['id'])
                num_following = acc['public_metrics']['following_count']
                num_followed_by = acc['public_metrics']['followers_count']
                this_acc_num_following.append(num_following)
                this_acc_num_followers.append(num_followed_by)

            this_acc_list = [this_acc for i in range(len(follow_list))]

        # Catch errors if not authorized to view user
        except Exception as e:
            print('Exception: issue retrieving follow lists for this account -- Skipping.')
            return curr_edgelist
            
            
        if 'next_token' in data['meta'].keys():    # get page token for next request
                next_page_token = data['meta']['next_token']
        
        # when all data recorded transition to next action
        else:
            next_page_token = None
            if action == 'followers':
                d = {'originating_id': follow_list,
                    'receiving_id': this_acc_list,
                    'originating_following_count': this_acc_num_following, 
                    'originating_follower_count': this_acc_num_followers}
                edges_to_add = pd.DataFrame(d)
                curr_edgelist = pd.concat([curr_edgelist, edges_to_add])

                action = 'following'

                # reset lists
                follow_list = []
                this_acc_num_following = []    
                this_acc_num_followers = []

            elif action == 'following':
                d = {'originating_id': this_acc_list,
                    'receiving_id': follow_list,
                    'receiving_following_count': this_acc_num_following, 
                    'receiving_follower_count': this_acc_num_followers}
                edges_to_add = pd.DataFrame(d)
                curr_edgelist = pd.concat([curr_edgelist, edges_to_add])

                next_page_token = None

                return curr_edgelist

In [21]:
def transition_to_next_separation_degree(curr_edgelist, degrees_of_separation):
    try:
        # Drop all nodes with followers or following more than threshold
        to_drop = (curr_edgelist.originating_following_count > thresh) | \
            (curr_edgelist.originating_follower_count > thresh) | \
            (curr_edgelist.receiving_following_count > thresh) | \
            (curr_edgelist.receiving_follower_count > thresh)
        curr_edgelist.drop(index = curr_edgelist[to_drop].index, inplace=True)

        # create list of all neighbors in curr_edgelist df
        unvisited_neighbors = list(curr_edgelist.originating_id.append(curr_edgelist.receiving_id).unique())

        # if any nodes are in visited set, remove
        [unvisited_neighbors.remove(i) for i in unvisited_neighbors if i in visited]

        print()
        print(f'Current Number of edges from starting node {degrees_of_separation}')
        print(f'Num unvisited neighbors: {len(unvisited_neighbors)}')
        print('--------------------')
        degrees_of_separation +=1 
    
        return curr_edgelist, degrees_of_separation, unvisited_neighbors
    
    
    except Exception as e:
        print(e)
        print('--------------------EXCEPTION--------------------')
        return curr_edgelist, degrees_of_separation, []
        
    

In [23]:
from tqdm.notebook import tqdm

In [30]:
cols=['originating_id', 'receiving_id', 
          'originating_following_count', 'originating_follower_count',
         'receiving_following_count', 'receiving_follower_count']

# Process human accounts 
small_bot_accs = small.loc[small.label == 'bot']

# Create Edgelists for each account in the filtered botometer dataset.
for starting_acc, label in zip(small_bot_accs.twitter_id, small_bot_accs.label):
    if starting_acc in already_processed:
        continue
    print(f'Starting node == {starting_acc}')
    # Initialization code for populating edgelist starting from node specified by "starting_acc"
    edgelist = pd.DataFrame(columns=cols)

    degrees_of_separation = 0

    visited = set()
    visited.add(starting_acc)

    edgelist = add_this_nodes_edgelist(starting_acc, edgelist) 
    edgelist, degrees_of_separation, unvisited_neighbors = transition_to_next_separation_degree(edgelist, degrees_of_separation)
    
    # iterate thru all unvisited neighbors with 2 degree of separation from starting (labeled) node
    # process df in chunks before concatenating
    while degrees_of_separation <= 1:
        curr_edgelist = pd.DataFrame(columns=cols)
        for i in tqdm(unvisited_neighbors):
            print(i)
            curr_edgelist = add_this_nodes_edgelist(i, curr_edgelist)
            visited.add(i)
        
        curr_edgelist, degrees_of_separation, unvisited_neighbors = transition_to_next_separation_degree(curr_edgelist, degrees_of_separation)
        edgelist = pd.concat([edgelist, curr_edgelist])

    # Save in directory
    fn = 'edgelist_2_edges_from_start_' + str(starting_acc) + '.pkl'
    path = str(label) + '/' + fn
    edgelist.to_pickle(path)
    
    # Add twitter id to already processed list
    already_processed.append(starting_acc)

Rate limit exceeded. Sleeping for 876 seconds.


Starting node == 300415996

Current Number of edges from starting node 0
Num unvisited neighbors: 136
--------------------


  0%|          | 0/136 [00:00<?, ?it/s]

1402529913987600384
846307598378811392
786582588559286272
125393322
767549297386659840
831777377000779780
813654995262316545
816530471819427840
195670049
771670290736701440
771138358152089601
765720415700299776
Exception: issue retrieving follow lists for this account -- Skipping.
759364968638578688
744894763212537856
730334058538897411


Rate limit exceeded. Sleeping for 893 seconds.


...

Rate limit exceeded. Sleeping for 892 seconds.


2905050283
2853061662
2501458801
2432933340
2885220101
2609572854
2543394024
2678475668
2895808668
2884364394
2431847953
2782610455
2887116732
2891379150
2836540543


Rate limit exceeded. Sleeping for 892 seconds.


1462161996
2804547912
2881412520


ReadTimeout: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=None)

In [ ]:
while degrees_of_separation <= 1:
    for i in tqdm(unvisited_neighbors):
        print(i)
        curr_edgelist = add_this_nodes_edgelist(i, curr_edgelist)
        visited.add(i)

    curr_edgelist, degrees_of_separation, unvisited_neighbors = transition_to_next_separation_degree(curr_edgelist, degrees_of_separation)
    edgelist = pd.concat([edgelist, curr_edgelist])

# Save in directory
fn = 'edgelist_2_edges_from_start_' + str(starting_acc) + '.pkl'
path = str(label) + '/' + fn
edgelist.to_pickle(path)

# Add twitter id to already processed list
already_processed.append(starting_acc)

  0%|          | 0/739 [00:00<?, ?it/s]

773168775407341570
Exception: issue retrieving follow lists for this account -- Skipping.
716959681030139904
1156276326682120193
90823705
1376106881433800705
1414239877051289605
309547500
1355079904014946307
1359360760388800515
721164661534515200
107120896
1161187506991198210
1277196752089518081
822366260155404288
1246773786692579328


Rate limit exceeded. Sleeping for 889 seconds.


882990031295492096
1081573212582887424
1316844980
1082513503498395649
1235500970945081344
3392056279
1219787325560897536
631534601
71687841
970294540673286145
1075799778271756288
1183970484943126528
824081837886582784
133657734
1102534073342259200


Rate limit exceeded. Sleeping for 887 seconds.


...

1648226161
Exception: issue retrieving follow lists for this account -- Skipping.
901457450
1634528527
1642989499
598910579
Exception: issue retrieving follow lists for this account -- Skipping.
1313789774
1598064588
1580101770
1591494900
1621527342
109514493
1533687750
1401680839
1571557662
1483585195
1409985205


Rate limit exceeded. Sleeping for 888 seconds.


1589260308
1583542519
1458791388
1585631120
1479358825
1497130200
742354056
2855710518
1287748128351719424
1301519604590796806
1259835651597299712
Exception: issue retrieving follow lists for this account -- Skipping.
1062580292135731200
2399977776
851695052778741760
2496004098


Rate limit exceeded. Sleeping for 886 seconds.


904658605692993536
1547416926
73883129
Exception: issue retrieving follow lists for this account -- Skipping.
2617211575
Exception: issue retrieving follow lists for this account -- Skipping.
